# Simple LNG Market

The importing countries try to minimize their costs for the imported LNG. The object of this model is to minimize total LNG costs while satisfying constraints on the supplies available at each of the exporting nodes, and satisfying demand requirements at each of the importerting nodes.

In [2]:
import shutil
import sys
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.close('all')

from IPython.display import display
from geopy.geocoders import Nominatim
from geopy import distance
from pyomo.environ import *

geolocator = Nominatim(user_agent="Your_Name")
pd.set_option('precision', 3)
MMBtu = 23.161 #1m³LNG = 23.161MMBtu 

## 1 Data 

### 1.1 Importing Nodes

In [3]:
# Data for importing and exporting countries from "BP Stats Review (2020)"

# "Import (MMBtu)" column consists of imports in m³ multiplied with conversion factor

Imp_Countries = pd.DataFrame({
    
    "Country" : ["Japan", "China", "South Korea", "India", "France", "Spain",
                 "UK", "Italy", "Turkey", "Belgium"],
    
     "Import (MMBtu)" : [105.5*10**9 * MMBtu, 84.8*10**9 * MMBtu, 55.6*10**9 * MMBtu, 32.9*10**9 * MMBtu,
                       22.9*10**9 * MMBtu, 21.9*10**9 * MMBtu, 18*10**9 * MMBtu, 13.5*10**9 * MMBtu,
                       12.9*10**9 * MMBtu, 7.2*10**9 * MMBtu],
    
    "Regasification Terminal" : ["Sodegaura", "Yancheng", "Incheon", "Dahej", "Dunkirk",
                     "Barcelona", "Isle of Grain", "Rovigo", "Marmara", "Zeebrugge"]
})

#Imp_Countries["Import (MMBtu)"] = (Imp_Countries["Import (MMBtu)"].astype(float)/1000000000).astype(str)


# getting the coordinates of largest regas terminals, not neccesarry anymore, since there is no suitable package
# for calculating sea distances :(

Imp_Countries["Location"] = Imp_Countries["Regasification Terminal"].apply(geolocator.geocode)
Imp_Countries["Point"]= Imp_Countries["Location"].apply(lambda loc: tuple(loc.point) if loc else None)
Imp_Countries[["Latitude", "Longitude", "Altitude"]] = pd.DataFrame(Imp_Countries["Point"].to_list(), index=Imp_Countries.index)

del Imp_Countries["Altitude"]
del Imp_Countries["Location"]
del Imp_Countries["Point"]


#center the columns, display() method doesn't work anymore, so not necessary for now
def pd_centered(df):
    return df.style.set_table_styles([
        {"selector": "th", "props": [("text-align", "center")]},
        {"selector": "td", "props": [("text-align", "center")]}])

display(pd_centered(Imp_Countries))
#Imp_Countries

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

### 1.2 Exporting Nodes

In [ ]:
# define DF with data for exporting countries (BP 2020)

Exp_Countries = pd.DataFrame({
    
    "Country" : ["Qatar", "Australia", "USA", "Russia", "Malaysia", "Nigeria",
                 "Trinidad & Tobago", "Algeria", "Indonesia","Oman"],
    
    "Export (MMBtu)" : [107.1*10**9 * MMBtu, 104.7*10**9 * MMBtu, 47.5*10**9 * MMBtu, 39.4*10**9 * MMBtu,
                        35.1*10**9 * MMBtu, 28.8*10**9 * MMBtu, 17.0*10**9 * MMBtu, 16.9*10**9 * MMBtu,
                        16.5*10**9 * MMBtu, 14.1*10**9 * MMBtu],
    
    "Break Eeven Price ($/MMBtu)" : [2.5, 6.5, 6.0, 4.5, 6.0, 4.08, 5.1, 2.5, 6.0, 3.5],
    
    #"Feedstock ($/MMBtu)" : [0.5, 3.0, 3.0, 0.5, 3.0, 2.2, 3.0, 0.5, 2.0, 1.5],

    #"Liquefaction ($/MMBtu)" : [2.0, 3.5, 3.0, 4.0, 3.0, 1.88, 2.1, 2.0, 4.0, 2.0],
    
    "Liquefaction Terminal" : ["Ras Laffan", "Gladstone", "Sabine Pass", "Sabetta",
                               "Bintulu", "Bonny Island", "Point Fortin",
                               " Bethioua", "Bontang", "Qalhat"],
})

#Exp_Countries["Export (Bcm)"] = (Exp_Countries["Export (Bcm)"].astype(float)/1000000000).astype(str)


# Getting the coordinates of liquefaction terminals, not neccesarry anymore, since there is no suitable package
# for calculating sea distances :(

Exp_Countries["Location"] = Exp_Countries["Liquefaction Terminal"].apply(geolocator.geocode)
Exp_Countries["Point"] = Exp_Countries["Location"].apply(lambda loc: tuple(loc.point) if loc else None)
Exp_Countries[["Latitude", "Longitude", "Altitude"]] = pd.DataFrame(Exp_Countries["Point"].to_list(), index=Exp_Countries.index)

# remove unnecessary columns
del Exp_Countries["Altitude"]
del Exp_Countries["Location"]
del Exp_Countries["Point"]

display(pd_centered(Exp_Countries))
#Exp_Countries

### 1.3 Sea distances among the importing and exporting nodes

In [6]:
# Define DF with distances among largest liquefaction and regas terminals in nautical miles
# In case of lack of route to the specific terminal, next largest port was taken
# sea-distances.org, searoutes.com

# if route over canal: distance rounded on 0 for suez and on 5 for panama, needed for later calculations

Distances = pd.DataFrame([(6512, 5846, 6161, 1301, 6240, 4650, 6260, 4380, 3460, 6270),
                          
                          (3861, 4134, 4313, 5918, 11650, 10060, 11670, 9790, 8870, 11680),
                          
                          (9205, 10075, 10005, 9645, 4881, 5206, 4897, 6354, 6341, 4908),
                          
                          (4929, 5596, 5671, 10160, 2494, 4252, 2511, 5546, 5481, 2477),
                          
                          (2276, 1656, 1998, 3231, 8980, 7390, 9000, 7120, 6200, 9010),
                          
                          (10752, 10088, 10406, 6988, 4287, 3824, 4309, 4973, 4961, 4321),
                          
                          (8885, 9755, 9665, 8370, 3952, 3926, 3974, 5074, 5062, 3984),
                          
                          (9590, 8930, 9240, 4720, 1519, 343, 1541, 1432, 1417, 1552),
                          
                          (2651, 2181, 2487, 3517, 9260, 7670, 9280, 7390, 6470, 9290),
                          
                          (6046, 5379, 5694, 853, 5760, 4180, 5790, 3900, 2980, 5800)

                         ],

                  index=["Qatar", "Australia", "USA", "Russia", "Malaysia", 
                         "Nigeria", "Trinidad & Tobago", "Algeria", "Indonesia"
                         ,"Oman"],

                  columns = ("Japan", "China", "South Korea", "India", "France",
                             "Spain", "UK", "Italy", "Turkey", "Belgium"))

display(pd_centered(Distances))
#Distances

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

### 1.4 LNG Carrier

To keep the model simple it is assumed that there is only one type of the LNG carrier with the following characteristics:

In [7]:
# define DF with lng carrier properties

LNG_Carrier = pd.DataFrame({"Capacity (m³)" : [160000], # average capacity in 2019 160000 (!!! https://www.rivieramm.com/opinion/opinion/lng-shipping-by-numbers-36027)
                            "Spot Charter Rate ($/day)" : [70000], # average spot charter rate in 2019
                            "Speed (knots/h)" : [17],
                            "Bunker (mt/d)" : [110], # !!! Does boil off reduce it and what's the ratio then??? 
                            "Bunkers Price ($/mt)" : [670], 
                            "Boil off" : [1/1000], # the daily guaranteed maximum boil-off or DGMB (0,1%)
                            "Heel" : [4/100]}) # 4% of the cargo is retained as the heel (LNG storage tanks need to be kept cool)

#print(LNG_Carrier_DF.loc[0, "Capacity (m³)"])
display(pd_centered(LNG_Carrier))
#LNG_Carrier

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

### 1.5 Additional Costs

- Port costs for 3 days: outbound port, destination port and one day in case it has to wait for loading/unloading
- Canal Fees (Suez or Panama Canal)
- Agents and broker fees: 2% of charter cost plus
- Insurance: 2,600 $/day

In [8]:
# define DF containing additional costs

Additional_Costs = pd.DataFrame({
    "Port Costs ($ for 3 days)" : [300000], 
    "Suez Canal Costs ($/Cargo)" : [1000000],  # 0.24 $/mmBtU
    "Panama Canal Costs ($/Cargo)" : [900000], # 0.21 $/mmBtu,                     
    "Insurance ($/day)" : [2600],
    "Fees (Percentage of Charter Costs)" : [2/100]})

display(pd_centered(Additional_Costs))
#Additional_Costs

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

## 2 Calculations

### 2.1 (Return) Voyage Times

In [9]:
Time = Distances / (LNG_Carrier.loc[0, "Speed (knots/h)"] * 12) # in days

display(pd_centered(Time))
# note to myself: it would be better to define a function which would compute the time!!!

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

### 2.2 Charter Costs

In [10]:
Charter_Costs = (Time + 2) * LNG_Carrier.loc[0, "Spot Charter Rate ($/day)"] # in $
display(pd_centered(Charter_Costs))

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

### 2.3 Fuel Costs

In [11]:
# this is arbitrary cost (it should be taken either price after relequifaction or destination hub price)
lng_cost = 5 # $/MMBtu 

Fuel_Costs = Time * LNG_Carrier.loc[0, "Boil off"] * 0.98 * LNG_Carrier.loc[0, "Capacity (m³)"] * lng_cost * MMBtu

# this is arbitrary cost (it should be taken either price after relequifaction or destination hub price)
lng_cost = 5 # $/MMBtu 

display(pd_centered(Fuel_Costs))

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

### 2.4 Agents and Broker Fees + Insurance

In [12]:
Fees_and_Insurance = (Time + 2) * Additional_Costs.loc[0, "Insurance ($/day)"] + Charter_Costs * Additional_Costs.loc[0, "Fees (Percentage of Charter Costs)"]
display(pd_centered(Fees_and_Insurance))

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

### 2.5 Suez/Panama Canal Fee

In [13]:
# create empty data frames for canal fees and port costs
Canal_Fees = pd.DataFrame(np.zeros((10,10)),
                   
                    index = ["Qatar", "Australia", "USA", "Russia", "Malaysia", 
                             "Nigeria", "Trinidad & Tobago", "Algeria", "Indonesia",
                             "Oman"],

                    columns = ("Japan", "China", "South Korea", "India", "France",
                               "Spain", "UK", "Italy", "Turkey", "Belgium"))

for i in Distances.index:
    for j in Distances.columns:
        if (((Distances.loc[i,j]) % 5 == 0) and ((Distances.loc[i,j]) %10 != 0)):
            Canal_Fees.loc[i,j] += Additional_Costs.loc[0, "Panama Canal Costs ($/Cargo)"]
        elif ((Distances.loc[i,j]) % 10 == 0):
            Canal_Fees.loc[i,j] += Additional_Costs.loc[0, "Suez Canal Costs ($/Cargo)"]
            
display(pd_centered(Canal_Fees))

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

### 2.6 Port Costs

In [14]:
# empty data frame for port costs
Port_Costs = pd.DataFrame(np.zeros((10,10)),
                   
                    index = ["Qatar", "Australia", "USA", "Russia", "Malaysia", 
                             "Nigeria", "Trinidad & Tobago", "Algeria", "Indonesia",
                             "Oman"],

                    columns = ("Japan", "China", "South Korea", "India", "France",
                               "Spain", "UK", "Italy", "Turkey", "Belgium"))

Port_Costs += Additional_Costs.loc[0, "Port Costs ($ for 3 days)"]
display(pd_centered(Port_Costs))

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

### 2.7 Total Costs of Transportation ($/MMBtu) 

In [15]:
Transport_Costs =  Charter_Costs + Fuel_Costs + Fees_and_Insurance + Canal_Fees + Port_Costs
#Transport_Costs

Transport_Costs_MMBtu = Transport_Costs / (0.94 * LNG_Carrier.loc[0, "Capacity (m³)"] * MMBtu) # 1-heel-boil_off = 0.94
display(pd_centered(Transport_Costs_MMBtu))

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

### 2.8 Total Costs in $/MMBtu

Total costs per unit of transported LNG = Feedstock Price + Liquefaction Costs + Total Costs of Transportation

In [16]:
# taken from EXP_Countries df

Breakeven = pd.DataFrame([(2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5),
                          
                        (6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5, 6.5),
                          
                        (6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0),
                          
                        (4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5, 4.5),
                          
                        (6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0),
                          
                        (4.08, 4.08, 4.08, 4.08, 4.08, 4.08, 4.08, 4.08, 4.08, 4.08),
                          
                        (5.1, 5.1, 5.1, 5.1, 5.1, 5.1, 5.1, 5.1, 5.1, 5.1),
                          
                        (2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5),
                          
                        (6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0),
                          
                        (3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5)

                       ],

                  index = ["Qatar", "Australia", "USA", "Russia", "Malaysia", "Nigeria",
                           "Trinidad & Tobago", "Algeria", "Indonesia","Oman"],

                  columns = ("Japan", "China", "South Korea", "India", "France",
                             "Spain", "UK", "Italy", "Turkey", "Belgium"))


#print(Breakeven.head(3))
Total_Costs = Breakeven + Transport_Costs_MMBtu
display(pd_centered(Total_Costs))

TypeError: Error trying to display values for object of type <class 'pandas.io.formats.style.Styler'>:
	Object does not support the 'display()' method

## 3 Pyomo Model

### 3.1 Data File

In [24]:
# Extract "Countries", "Exports" and "Imports" from "Exp_Countries" and "Imp_Countries" dfs
# and create dictionaries

Supply = Exp_Countries.set_index("Country")["Export (MMBtu)"].to_dict()
#print(Supply)

Demand = Imp_Countries.set_index("Country")["Import (MMBtu)"].to_dict()
#print(Demand)

# I would really like to get "Cost" dictionary straight from the "Total_Costs" df, since doing it 
# this way is real pain in the ...
Cost = {
    ("Japan", "Qatar"): 3.473,
    ("China", "Qatar"): 3.387,
    ("South Korea", "Qatar"): 3.428,
    ("India", "Qatar"): 2.797,
    ("France", "Qatar"): 3.725,
    ("Spain", "Qatar"): 3.519,
    ("UK", "Qatar"): 3.728,
    ("Italy", "Qatar"): 3.484,
    ("Turkey", "Qatar"): 3.364,
    ("Belgium", "Qatar"): 3.729,
    ("Japan", "Australia"): 7.129,
    ("China", "Australia"): 7.165,
    ("South Korea", "Australia"): 7.188,
    ("India", "Australia"): 7.396,
    ("France", "Australia"): 8.427,
    ("Spain", "Australia"): 8.220,
    ("UK", "Australia"): 8.429,
    ("Italy", "Australia"): 8.185,
    ("Turkey", "Australia"): 8.066,
    ("Belgium", "Australia"): 8.430,
    ("Japan", "USA"): 7.581,
    ("China", "USA"): 7.694,
    ("South Korea", "USA"): 7.685,
    ("India", "USA"): 7.638,
    ("France", "USA"): 6.762,
    ("Spain", "USA"): 6.804,
    ("UK", "USA"): 6.764,
    ("Italy", "USA"): 6.953,
    ("Turkey", "USA"): 6.951,
    ("Belgium", "USA"): 6.765,
    ("Japan", "Russia"): 5.268,
    ("China", "Russia"): 5.354,
    ("South Korea", "Russia"): 5.364,
    ("India", "Russia"): 6.233,
    ("France", "Russia"): 4.952,
    ("Spain", "Russia"): 5.180,
    ("UK", "Russia"): 4.954,
    ("Italy", "Russia"): 5.348,
    ("Turkey", "Russia"): 5.339,
    ("Belgium", "Russia"): 4.950,
    ("Japan", "Malaysia"): 6.424,
    ("China", "Malaysia"): 6.343,
    ("South Korea", "Malaysia"): 6.388,
    ("India", "Malaysia"): 6.548,
    ("France", "Malaysia"): 7.580,
    ("Spain", "Malaysia"): 7.374,
    ("UK", "Malaysia"): 7.583,
    ("Italy", "Malaysia"): 7.339,
    ("Turkey", "Malaysia"): 7.220,
    ("Belgium", "Malaysia"): 7.584,
    ("Japan", "Nigeria"): 5.603,
    ("China", "Nigeria"): 5.517,
    ("South Korea", "Nigeria"): 5.558,
    ("India", "Nigeria"): 5.115,
    ("France", "Nigeria"): 4.765,
    ("Spain", "Nigeria"): 4.705,
    ("UK", "Nigeria"): 4.767,
    ("Italy", "Nigeria"): 4.854,
    ("Turkey", "Nigeria"): 4.852,
    ("Belgium", "Nigeria"): 4.769,
    ("Japan", "Trinidad & Tobago"): 6.639,
    ("China", "Trinidad & Tobago"): 6.752,
    ("South Korea", "Trinidad & Tobago"): 6.740,
    ("India", "Trinidad & Tobago"): 6.601,
    ("France", "Trinidad & Tobago"): 5.741,
    ("Spain", "Trinidad & Tobago"): 5.738,
    ("UK", "Trinidad & Tobago"): 5.744,
    ("Italy", "Trinidad & Tobago"): 5.887,
    ("Turkey", "Trinidad & Tobago"): 5.885,
    ("Belgium", "Trinidad & Tobago"): 5.745,
    ("Japan", "Algeria"): 4.159,
    ("China", "Algeria"): 4.074,
    ("South Korea", "Algeria"): 4.114,
    ("India", "Algeria"): 3.528,
    ("France", "Algeria"): 2.826,
    ("Spain", "Algeria"): 2.673,
    ("UK", "Algeria"): 2.828,
    ("Italy", "Algeria"): 2.814,
    ("Turkey", "Algeria"): 2.812,
    ("Belgium", "Algeria"): 2.830, 
    ("Japan", "Indonesia"): 6.472,
    ("China", "Indonesia"): 6.411,
    ("South Korea", "Indonesia"): 6.451,
    ("India", "Indonesia"): 6.585,
    ("France", "Indonesia"): 7.617,
    ("Spain", "Indonesia"): 7.410,
    ("UK", "Indonesia"): 7.619,
    ("Italy", "Indonesia"): 7.374,
    ("Turkey", "Indonesia"): 7.255,
    ("Belgium","Indonesia"): 7.620,
    ("Japan", "Oman"): 4.413,
    ("China", "Oman"): 4.326,
    ("South Korea", "Oman"): 4.367,
    ("India", "Oman"): 3.739,
    ("France", "Oman"): 4.663,
    ("Spain", "Oman"): 4.458,
    ("UK", "Oman"): 4.667,
    ("Italy", "Oman"): 4.421,
    ("Turkey", "Oman"): 4.302,
    ("Belgium", "Oman"): 4.668    
}

#print(Cost)

### 3.1 Model File

In [25]:
# Step 0: Create an instance of the model
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

# Step 1: Define index sets
CUS = list(Demand.keys()) # Customer
SRC = list(Supply.keys()) # Source

# Step 2: Define the decision 
model.x = Var(CUS, SRC, domain = NonNegativeReals)

# Step 3: Define Objective
model.Cost = Objective(
    expr = sum([Cost[c,s]*model.x[c,s] for c in CUS for s in SRC]),
    sense = minimize)

# Step 4: Constraints
model.src = ConstraintList()
for s in SRC:
    model.src.add(sum([model.x[c,s] for c in CUS]) <= Supply[s])
        
model.dmd = ConstraintList()
for c in CUS:
    model.dmd.add(sum([model.x[c,s] for s in SRC]) == Demand[c])
    
results = SolverFactory('glpk').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 44639429781300.0
  Upper bound: 44639429781300.0
  Number of objectives: 1
  Number of constraints: 21
  Number of variables: 101
  Number of nonzeros: 201
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.02081131935119629
# ----------------------------------------------------------
#   Solution Information
# ----------

## 4 Solution

In [44]:
#for c in CUS:
#    for s in SRC:
#        print(c, s, model.x[c,s]() / MMBtu / 1000000000)

In [26]:
if 'ok' == str(results.Solver.status):
    #print("Total Costs = ",model.Cost(), "$")
    print("\nShipping Table:")
    for s in SRC:
        for c in CUS:
            if model.x[c,s]() > 0:
                print("Ship from ", s," to ", c, ":", model.x[c,s]() / MMBtu / 1000000000, "bcm")
else:
    print("No Valid Solution Found")


Shipping Table:
Ship from  Qatar  to  China : 35.6 bcm
Ship from  Qatar  to  South Korea : 38.6 bcm
Ship from  Qatar  to  India : 32.9 bcm
Ship from  Australia  to  Japan : 66.1 bcm
Ship from  Australia  to  South Korea : 0.5 bcm
Ship from  USA  to  France : 8.499999999999998 bcm
Ship from  USA  to  UK : 18.0 bcm
Ship from  USA  to  Belgium : 7.2 bcm
Ship from  Russia  to  Japan : 39.4 bcm
Ship from  Malaysia  to  China : 35.1 bcm
Ship from  Nigeria  to  Spain : 19.3 bcm
Ship from  Nigeria  to  Italy : 9.5 bcm
Ship from  Trinidad & Tobago  to  France : 14.4 bcm
Ship from  Trinidad & Tobago  to  Spain : 2.6 bcm
Ship from  Algeria  to  Italy : 4.0 bcm
Ship from  Algeria  to  Turkey : 12.9 bcm
Ship from  Indonesia  to  South Korea : 16.499999999999996 bcm
Ship from  Oman  to  China : 14.1 bcm


In [43]:
print("\nShipping Table:")
for c in CUS:
    for s in SRC:
        if model.x[c,s]() > 0:
                print("Ship from ", s," to ", c, ":", model.x[c,s]() / MMBtu / 1000000000, "bcm")


Shipping Table:
Ship from  Australia  to  Japan : 66.1 bcm
Ship from  Russia  to  Japan : 39.4 bcm
Ship from  Qatar  to  China : 35.6 bcm
Ship from  Malaysia  to  China : 35.1 bcm
Ship from  Oman  to  China : 14.1 bcm
Ship from  Qatar  to  South Korea : 38.6 bcm
Ship from  Australia  to  South Korea : 0.5 bcm
Ship from  Indonesia  to  South Korea : 16.499999999999996 bcm
Ship from  Qatar  to  India : 32.9 bcm
Ship from  USA  to  France : 8.499999999999998 bcm
Ship from  Trinidad & Tobago  to  France : 14.4 bcm
Ship from  Nigeria  to  Spain : 19.3 bcm
Ship from  Trinidad & Tobago  to  Spain : 2.6 bcm
Ship from  USA  to  UK : 18.0 bcm
Ship from  Nigeria  to  Italy : 9.5 bcm
Ship from  Algeria  to  Italy : 4.0 bcm
Ship from  Algeria  to  Turkey : 12.9 bcm
Ship from  USA  to  Belgium : 7.2 bcm
